In [ ]:
import logging
import time
import lotr

logging.basicConfig(level=logging.INFO, format='%(asctime)s: %(message)s')

In [ ]:
conf = lotr.read_conf()

In [ ]:
timestamp = time.time()
lotr.reset_project_folders(conf)
lotr.download_sheet(conf)
sets = lotr.get_sets(conf)

changes = False
for set_data in sets:
    set_id, set_name, set_row = set_data
    if conf['octgn']:
        lotr.generate_octgn_xml(set_id, set_name, set_row)

    lotr.copy_custom_images(conf, set_id, set_name)
    for lang in conf['languages']:
        lotr.generate_xml(conf, set_id, set_name, set_row, lang)
        lotr.update_xml(conf, set_id, set_name, lang)
        new_hash, old_hash = lotr.calculate_hashes(set_id, set_name, lang)
        if new_hash != old_hash:
            changes = True

        lotr.copy_raw_images(conf, set_id, set_name, lang)
        lotr.copy_xml(set_id, set_name, lang)

if changes:
    if conf['octgn'] and conf['octgn_destination_path']:
        lotr.copy_octgn_outputs(conf)

    lotr.create_project()
else:
    logging.info('No changes since the last run, skipping')

logging.info('Done (%ss)', round(time.time() - timestamp, 3))

### Strange Eons Part:

1. Open `setGenerator.seproject` in Strange Eons.
2. Run `Script/makeCards` script by double clicking it.
3. Once completed, close Strange Eons (wait until it finished packing the project).

In [ ]:
timestamp = time.time()
for set_data in sets:
    set_id, set_name, _ = set_data
    for lang in conf['languages']:
        skip_set, skip_ids = lotr.get_skip_info(set_id, set_name, lang)
        if skip_set:
            logging.info('[%s, %s] No changes since the last run,'
                         ' skipping', set_name, lang)
            continue

        if conf['nobleed'][lang]:
            lotr.generate_png300_nobleed(conf, set_id, set_name, lang, skip_ids)

        if 'db' in conf['outputs'][lang]:
            lotr.generate_png300_db(conf, set_id, set_name, lang, skip_ids)
            lotr.generate_db(set_id, set_name, lang)

        if 'octgn' in conf['outputs'][lang]:
            lotr.generate_png300_octgn(set_id, set_name, lang, skip_ids)
            lotr.generate_octgn(set_id, set_name, lang)

        if 'pdf' in conf['outputs'][lang]:
            lotr.generate_png300_pdf(conf, set_id, set_name, lang, skip_ids)
            lotr.generate_pdf(conf, set_id, set_name, lang)

        if 'makeplayingcards' in conf['outputs'][lang]:
            lotr.generate_png800_bleedmpc(conf, set_id, set_name, lang, skip_ids)
            lotr.generate_mpc(conf, set_id, set_name, lang)

        if 'drivethrucards' in conf['outputs'][lang]:
            lotr.generate_jpg300_bleeddtc(conf, set_id, set_name, lang, skip_ids)
            lotr.generate_dtc(conf, set_id, set_name, lang)

logging.info('Done (%ss)', round(time.time() - timestamp, 3))

Now there should be the following outputs:

- `Output/OCTGN/<set name>/`: `<octgn id>/set.xml` and `<set name>.<language>.o8c` image packs for OCTGN (300 dpi PNG).
- `Output/DB/<set name>.<language>/`: 300 dpi PNG images for general purposes.
- `Output/PDF/<set name>.<language>/`: PDF files in `A4` and `letter` format for home printing.
- `Output/DriveThruCards/<set name>.<language>/`: `zip` and `7z` archives of 300 dpi JPG images to be printed on DriveThruCards.com.
- `Output/MakePlayingCards/<set name>.<language>/`: `zip` and `7z` archives of 800 dpi PNG images to be printed on MakePlayingCards.com.

Additionally, if you specified `octgn_destination_path`, OCTGN outputs will be copied there.